<a href="https://colab.research.google.com/github/shivamsouravjha/-PetFinder.my-Adoption-Prediction-/blob/master/Petfinder_Using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import os
from keras.layers import Dense,BatchNormalization
from sklearn.model_selection import train_test_split

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
trainX = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/test.csv')
print(trainX.shape)
print(test.shape)
trainX.head()


(14993, 24)
(3972, 23)


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [5]:
for col in trainX.columns:
    if trainX[col].dtype=='object':
        trainX.drop([col],axis=1,inplace=True)

In [6]:
for col in test.columns:
    if test[col].dtype=='object':
        test.drop([col],axis=1,inplace=True)

In [7]:
trainY = pd.DataFrame()
trainY['AdoptionSpeed'] = trainX['AdoptionSpeed']
trainX=trainX.drop(['AdoptionSpeed','State',	'VideoAmt'	,'PhotoAmt'],axis=1)
test=test.drop(['State','VideoAmt'	,'PhotoAmt'],axis=1)

In [8]:
print(trainX.shape)
trainX.head()


(14993, 16)


,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0


In [9]:
print(test.shape)
test.head()


(3972, 16)


,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee
0,2,8,266,266,1,2,6,7,1,1,1,1,2,1,2,0
1,2,36,285,264,2,1,4,7,2,3,1,1,1,2,1,0
2,2,2,265,0,1,6,7,0,2,2,1,1,2,1,1,200
3,1,12,307,0,2,2,5,0,2,2,1,1,1,1,2,0
4,2,12,265,0,1,2,3,7,2,2,1,1,1,1,1,0


In [10]:
#test_image = os.listdir('../input/petfinder-adoption-prediction/test_images')
#train_image = os.listdir('../input/petfinder-adoption-prediction/train_images')

In [11]:
#def display_samples(columns=4, rows=3):
 #   fig=plt.figure(figsize=(5*columns, 3*rows))
  #  for i in range(columns*rows):
   #     image_path = train_image[i]
    #    img = cv2.imread(f'../input/petfinder-adoption-prediction/train_images/{image_path}')
     #   fig.add_subplot(rows, columns, i+1)
      #  plt.imshow(img)

#display_samples()


In [1]:
from keras.models import Sequential

In [12]:
model  = Sequential()

In [13]:
trainX.shape

(14993, 16)

In [14]:
model.add(Dense(4096,activation= 'relu',input_shape=(16,))) #dense layer 1
model.add(tf.keras.layers.BatchNormalization()) #BachNorm
model.add(tf.keras.layers.Dropout(0.25)) #Dropout
model.add(Dense(2048,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout
model.add(Dense(1024,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout
model.add(Dense(512,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.35)) #Dropout
model.add(Dense(256,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout

model.add(Dense(128,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout

model.add(Dense(64,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.35)) #Dropout
model.add(BatchNormalization())

model.add(Dense(32,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.25)) #Dropout

model.add(Dense(16,activation= 'relu'))
model.add(tf.keras.layers.Dropout(0.15)) #Dropout

model.add(Dense(5,activation= 'softmax'))  

In [15]:
from keras.optimizers import RMSprop,Adam,SGD,Adagrad,Adadelta,Adamax,Nadam

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.99)
# Compile the model
model.compile(optimizer = optimizer , loss = 'categorical_crossentropy', metrics=["accuracy"])


In [16]:
trainX =trainX.values

In [17]:
trainY=trainY.values

In [18]:
valx,testx,valy,testy = train_test_split(trainX,trainY,test_size=0.25)

In [19]:
testy = tf.keras.utils.to_categorical(testy, 5)
valy = tf.keras.utils.to_categorical(valy, 5)

In [20]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.37):
            print("\nReached 40.0% accuracy so cancelling training!")
            self.model.stop_training = True


In [21]:
callback = myCallback()

In [22]:
model.fit(testx,testy,validation_data =(valx,valy),batch_size=750,epochs=256,callbacks = [callback])

Epoch 1/256
5/5 [==============================] - 0s 85ms/step - loss: 1.8302 - accuracy: 0.2121 - val_loss: 2.7652 - val_accuracy: 0.2541
Epoch 2/256
5/5 [==============================] - 0s 42ms/step - loss: 1.7208 - accuracy: 0.2150 - val_loss: 1.8939 - val_accuracy: 0.2335
Epoch 3/256
5/5 [==============================] - 0s 35ms/step - loss: 1.6565 - accuracy: 0.2577 - val_loss: 1.6265 - val_accuracy: 0.2223
Epoch 4/256
5/5 [==============================] - 0s 36ms/step - loss: 1.6149 - accuracy: 0.2587 - val_loss: 1.5721 - val_accuracy: 0.2208
Epoch 5/256
5/5 [==============================] - 0s 35ms/step - loss: 1.5862 - accuracy: 0.2675 - val_loss: 1.6270 - val_accuracy: 0.2179
Epoch 6/256
5/5 [==============================] - 0s 34ms/step - loss: 1.5646 - accuracy: 0.2665 - val_loss: 1.6164 - val_accuracy: 0.2012
Epoch 7/256
5/5 [==============================] - 0s 35ms/step - loss: 1.5476 - accuracy: 0.2710 - val_loss: 1.5746 - val_accuracy: 0.2111
Epoch 8/256
5/5 [===

In [23]:
scores = model.evaluate(valx, valy, verbose=0)
print("{}: {:.2f}%".format("accuracy", scores[1]*100))

accuracy: 32.45%


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4096)              69632     
_________________________________________________________________
batch_normalization (BatchNo (None, 4096)              16384     
_________________________________________________________________
dropout (Dropout)            (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              8390656   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0

In [25]:
print(test.shape)

(3972, 16)


In [26]:
test.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee
0,2,8,266,266,1,2,6,7,1,1,1,1,2,1,2,0
1,2,36,285,264,2,1,4,7,2,3,1,1,1,2,1,0
2,2,2,265,0,1,6,7,0,2,2,1,1,2,1,1,200
3,1,12,307,0,2,2,5,0,2,2,1,1,1,1,2,0
4,2,12,265,0,1,2,3,7,2,2,1,1,1,1,1,0


In [27]:
prc =model.predict(test)


In [28]:
print(prc)

[[0.06714991 0.23652971 0.2497142  0.17980394 0.26680225]
 [0.02090253 0.25154784 0.3582338  0.21574968 0.1535661 ]
 [0.02376455 0.388972   0.33573505 0.19726813 0.05426028]
 ...
 [0.05703691 0.25712925 0.28498718 0.18650967 0.21433693]
 [0.00682585 0.18411544 0.38972697 0.24380766 0.17552406]
 [0.10539752 0.27713653 0.2663049  0.17368065 0.1774803 ]]


In [29]:
prc = prc.argmax(axis=1)

In [30]:
print(prc)

[4 2 1 ... 2 2 1]


In [32]:
sample = pd.read_csv('/content/drive/My Drive/Colab Notebooks/sample_submission.csv')

In [33]:
sample.AdoptionSpeed = prc

In [34]:
sample.head(50)

,PetID,AdoptionSpeed
0,e2dfc2935,4
1,f153b465f,2
2,3c90f3f54,1
3,e02abc8a3,1
4,09f0df7d1,1
5,0487529d4,2
6,bae7c4b1c,2
7,548bcf206,1
8,0f82cea1e,1
9,a3787f15e,4


In [35]:
sample.to_csv('submission.csv', index=False)